Imports


In [5]:
from preprocess import prepare_loan_train_test, preprocess_loan_features
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from evaluate import plot_confusion_matrix, plot_feature_importance, plot_parameter_sensitivity, get_results, get_cross_validation_scores
from scipy.stats import randint, loguniform
import pandas as pd
import matplotlib.pylab as plt
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
import numpy as np
from scipy.stats import randint, uniform
from tensorflow.keras.optimizers import Adam
import time

In [6]:
# df to save random search times
index_name = 'model_name'
column_name = 'randsearch_time'

df_times = pd.DataFrame(
    index = pd.Index([], name=index_name),
    columns = [column_name],
    dtype = np.float64
)

## Base model with using SMOTE

In [9]:
training_data = pd.read_csv("loan-10k.lrn.csv") 
training_data = preprocess_loan_features(training_data)
X_train, X_val, y_train, y_val = prepare_loan_train_test(training_data, scale=True)

# Use SMOTE
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)

target_names = ["A","B", "C", "D", "E", "F", "G"]

In [10]:
early_stopper = EarlyStopping(
    monitor='val_loss', 
    patience=10,        
    restore_best_weights=True
)

# required for crossval
def build_model(input_dim):
    model = Sequential([
        Input(shape = (input_dim,)),
        Dense(100, activation='relu'),
        Dense(2, activation='softmax') # Using  sparse_categorical_crossentropy as the labels aren't one-hot encoded
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy', 
        metrics=['accuracy']
    )
    return model

# Required for crossval
keras_clf = KerasClassifier(
    model=build_model,
    model__input_dim=X_train.shape[1], 
    epochs=500,
    batch_size=32,
    callbacks=[early_stopper],
    validation_split=0.16,
    verbose=0 
)

base_model = build_model(X_train.shape[1])
history = base_model.fit(
    X_train, 
    y_train, 
    epochs=500,
    batch_size=32,
    validation_split=0.15,
    callbacks=[early_stopper],
    verbose=0 
)

2025-11-09 20:31:37.375707: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


ValueError: Invalid dtype: object